In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pygetwindow as gw
import pyautogui
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 사용 컴퓨터와 위치에 따라 초기 지도위치가 다르다.

url = 'https://map.kakao.com/'
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get(url)

# 윈도우창 위치, 크기 조정
chrome = gw.getWindowsWithTitle('카카오맵')[0]

chrome.moveTo(0, 0)
chrome.resizeTo(1395, 1000)

# 교육장을 검색해서 가운데로 위치하게 하자
# 창이 열리고 바로 검색을 하면 검색 버튼을 찾지 못한다.
# 때문에 모션을 실행 후 검색

pyautogui.PAUSE = 0.5
pyautogui.click(750,750)
pyautogui.click(750,750)
pyautogui.press('+')

search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
search.clear()
search.send_keys('청년취업사관학교 영등포캠퍼스')

sch_btn = driver.find_element(By.CSS_SELECTOR, 'div.box_searchbar > button')
sch_btn.click()

pyautogui.click(750,750)
pyautogui.press('+')

# 현 지도 내 장소검색 클릭
sch_btn = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.bounds')
sch_btn.click()

# 지도 위치를 고정하기 위해서 드래그 위치를 지정
# [[시작좌표],[끝좌표]] 형식으로 지정
# 드래그 위치는 화면 해상도에 따라 다를 수 있다.
drag_position = [[[1000,600],[1610,1130]],[[1225,600],[0,600]],[[800,950],[800,-100]],[[420,600],[1645,600]]]

# 추출된 데이터 저장을 위한 DataFrame 생성
res_lst = pd.DataFrame(columns=['가맹점명','주소','영업시간', '분류','점수','건수'])

cnt = 0
for lab, po_lst in enumerate(drag_position):
    # 드래그 작업
    pyautogui.moveTo(po_lst[0])
    pyautogui.dragTo(po_lst[1],duration=0.25)
    pyautogui.sleep(0.5)

    # print(lab)  
    if lab == 0:
        # 드래그로 위치를 잡은 후 식당 검색
        search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
        search.clear()
        search.send_keys('식당')
        sch_btn = driver.find_element(By.CSS_SELECTOR, 'div.box_searchbar > button')
        sch_btn.click()
        pyautogui.sleep(0.5)
        search = driver.find_element(By.CSS_SELECTOR, '#info\.search\.place\.more')
        search.click()
        pyautogui.sleep(0.5)

    off = 0
    while off == 0:
        
        for page in range(1,6):
            
            # 페이지 번호 클릭
            # 페이지 번호가 없다면 off를 1로 변경하여 다음 드래그 위치로 이동 
            try:
                search = driver.find_element(By.CSS_SELECTOR, f'#info\.search\.page\.no{page}')
                search.click()
                pyautogui.sleep(0.5)
            except:
                off = 1
                break
        
            # 필요 정보 추출
            for num in range(1,16):
                search_name = driver.find_element(By.CSS_SELECTOR, '#info\.noPlace > div')
                
                if search_name.text == '':
                    try:
                        search_name = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.head_item.clickArea > strong > a.link_name')
                        res_lst.loc[cnt,"가맹점명"] = search_name.text
                    except:
                        res_lst.loc[cnt,"가맹점명"] = 'error'

                    try:
                        search_add = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.info_item > div.addr > p:nth-child(1)')
                        res_lst.loc[cnt,"주소"] = search_add.text
                    except:
                        res_lst.loc[cnt,"주소"] = 'error'

                    try:
                        search_add = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.info_item > div.openhour > p > a')
                        res_lst.loc[cnt,"영업시간"] = search_add.text
                    except:
                        res_lst.loc[cnt,"영업시간"] = 'error'

                    try:
                        search_add = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.head_item.clickArea > span')
                        res_lst.loc[cnt,"분류"] = search_add.text
                    except:
                        res_lst.loc[cnt,"분류"] = 'error'

                    try:
                        star = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.rating.clickArea > span.score > em')
                        no_review = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.rating.clickArea > span.txt_blind')
                        if no_review.text == "" and star.text != "":
                            res_lst.loc[cnt,"점수"] = star.text
                        elif no_review.text != "" and star.text == "":
                            res_lst.loc[cnt,"점수"] = no_review.text
                    except:
                        res_lst.loc[cnt,"점수"] = 'error'

                    try:
                        num = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.rating.clickArea > span.score > a')
                        res_lst.loc[cnt,"건수"] = num.text.replace('건','')
                    except:
                        res_lst.loc[cnt,"건수"] = 'error'

                cnt += 1

            # 페이지가 5페이지라면 다음 페이지로 이동
            # 다음 페이지가 없으면 off를 1로 변경하여 다음 드래그 위치로 이동
            if page == 5:
                search = driver.find_element(By.CSS_SELECTOR, '#info\.search\.page\.next')
                if search.get_attribute("class") == 'next':
                    search.click()
                    pyautogui.sleep(0.5)
                elif search.get_attribute("class") == 'next disabled':
                    off = 1

driver.quit()

0
1
2
3


In [13]:
# 겹치는 지도로 인한 중복 식당 제거
res_lst.drop_duplicates(inplace=True)
res_lst

,가맹점명,주소,영업시간,분류,점수,건수
0,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,영업시간 매일 11:00 ~ 21:20 · 휴게시간 매일 15:30 ~ 16:30,"해물,생선",4.1,43
1,타이산,서울 영등포구 선유로13길 25 에이스하이테크시티2 지식산업센터 116호,,중국요리,3.8,9
2,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층",영업시간 매일 11:30 ~ 22:00,"육류,고기",3.5,27
3,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,영업시간 월~토 11:00 ~ 21:00 · 휴게시간 월~토 15:00 ~ 17:00,"해물,생선",4.5,2
4,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,영업시간 화~토 12:00 ~ 22:00 · 휴게시간 화~토 15:00 ~ 17:00,양식,4.1,23
...,...,...,...,...,...,...
502,더문,서울 영등포구 선유로 27,,술집,0.0,0
503,그린스푼,서울 영등포구 선유로3길 10 하우스디비즈 지하1층 B101호,,한식,0.0,0
504,먹태랑노가리,서울 영등포구 도림로 461,,"호프,요리주점",0.0,0
505,골든푸드코리아,서울 영등포구 도림로 456 상가 101동 105호,,한식,0.0,0


In [14]:
# 인덱스 초기화
res_lst.reset_index(drop=True,inplace=True)
res_lst

,가맹점명,주소,영업시간,분류,점수,건수
0,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,영업시간 매일 11:00 ~ 21:20 · 휴게시간 매일 15:30 ~ 16:30,"해물,생선",4.1,43
1,타이산,서울 영등포구 선유로13길 25 에이스하이테크시티2 지식산업센터 116호,,중국요리,3.8,9
2,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층",영업시간 매일 11:30 ~ 22:00,"육류,고기",3.5,27
3,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,영업시간 월~토 11:00 ~ 21:00 · 휴게시간 월~토 15:00 ~ 17:00,"해물,생선",4.5,2
4,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,영업시간 화~토 12:00 ~ 22:00 · 휴게시간 화~토 15:00 ~ 17:00,양식,4.1,23
...,...,...,...,...,...,...
415,더문,서울 영등포구 선유로 27,,술집,0.0,0
416,그린스푼,서울 영등포구 선유로3길 10 하우스디비즈 지하1층 B101호,,한식,0.0,0
417,먹태랑노가리,서울 영등포구 도림로 461,,"호프,요리주점",0.0,0
418,골든푸드코리아,서울 영등포구 도림로 456 상가 101동 105호,,한식,0.0,0


In [15]:
res_lst

,가맹점명,주소,영업시간,분류,점수,건수
0,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,영업시간 매일 11:00 ~ 21:20 · 휴게시간 매일 15:30 ~ 16:30,"해물,생선",4.1,43
1,타이산,서울 영등포구 선유로13길 25 에이스하이테크시티2 지식산업센터 116호,,중국요리,3.8,9
2,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층",영업시간 매일 11:30 ~ 22:00,"육류,고기",3.5,27
3,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,영업시간 월~토 11:00 ~ 21:00 · 휴게시간 월~토 15:00 ~ 17:00,"해물,생선",4.5,2
4,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,영업시간 화~토 12:00 ~ 22:00 · 휴게시간 화~토 15:00 ~ 17:00,양식,4.1,23
...,...,...,...,...,...,...
415,더문,서울 영등포구 선유로 27,,술집,0.0,0
416,그린스푼,서울 영등포구 선유로3길 10 하우스디비즈 지하1층 B101호,,한식,0.0,0
417,먹태랑노가리,서울 영등포구 도림로 461,,"호프,요리주점",0.0,0
418,골든푸드코리아,서울 영등포구 도림로 456 상가 101동 105호,,한식,0.0,0


In [16]:
# error 식당 제거
res_lst_drop = res_lst.drop(res_lst.loc[res_lst['가맹점명']=='error'].index)
res_lst_drop.reset_index(drop=True,inplace=True)
res_lst_drop

,가맹점명,주소,영업시간,분류,점수,건수
0,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,영업시간 매일 11:00 ~ 21:20 · 휴게시간 매일 15:30 ~ 16:30,"해물,생선",4.1,43
1,타이산,서울 영등포구 선유로13길 25 에이스하이테크시티2 지식산업센터 116호,,중국요리,3.8,9
2,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층",영업시간 매일 11:30 ~ 22:00,"육류,고기",3.5,27
3,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,영업시간 월~토 11:00 ~ 21:00 · 휴게시간 월~토 15:00 ~ 17:00,"해물,생선",4.5,2
4,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,영업시간 화~토 12:00 ~ 22:00 · 휴게시간 화~토 15:00 ~ 17:00,양식,4.1,23
...,...,...,...,...,...,...
414,더문,서울 영등포구 선유로 27,,술집,0.0,0
415,그린스푼,서울 영등포구 선유로3길 10 하우스디비즈 지하1층 B101호,,한식,0.0,0
416,먹태랑노가리,서울 영등포구 도림로 461,,"호프,요리주점",0.0,0
417,골든푸드코리아,서울 영등포구 도림로 456 상가 101동 105호,,한식,0.0,0


In [17]:
# 엑셀로 저장
res_lst.to_excel("주변식당목록_error.xlsx",index=False)
res_lst_drop.to_excel("주변식당목록.xlsx",index=False)